# 데이터 구조 이해 & SQL 스키마 설계
- JSON 데이터 로드 및 컬럼 확인 (pandas)
- 결측치·중복 데이터 확인/정제
- SQL DB 스키마 설계 (Supplements, Ingredients, Supplement_Ingredients, Symptoms_Ingredients)• 증상–성분 매핑 테이블 초안 작성

- json 파일 데이터 개수: 1241개, 23열 ( )

In [ ]:
import pandas as pd
import numpy as np

In [ ]:

# 1) 로드
fp = '/Users/gim-yujin/Desktop/pjt_personal_agent/영양소 데이터/JSON 파일 /iherb_data_uk_data_2022_12.json'
df = pd.read_json(fp, orient='records')   # 파일이 리스트 of dicts 여야 정상
# 2) 전체 컬럼 확인
print(df.shape)
print(df.columns.tolist())
# 3) 샘플 확인
display(df.head())
# 4) 기본 타입 정리
df['scraped_at'] = pd.to_datetime(df['scraped_at'], dayfirst=True, errors='coerce')

In [ ]:
print("##컬럼명 목록")
print(df.columns)
print("-" * 50)
# 컬럼별 결측치 개수 확인 
print("##컬럼별 결측치 개수 확인(공백 문자열이 결측치로 합산이 안되어 모두 0으로 표기됨)")
print(df.isnull().sum())

In [ ]:
df[['Supplement Facts']]

### 결측치 확인
- 결측값이 존재하는 컬럼 및 개수
- Category 2            4
- Category 3          869
- ingredients          61
- Supplement Facts    530 (추후에 성분을 참고하여 채워 넣을 예정, 보충정보(영양성분))

In [ ]:
# (선택) 모든 컬럼에 대해 한 번에 적용할 수도 있습니다.
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
print("\n## 전체 데이터의 실제 결측치 수")
print(df.isnull().sum())

In [ ]:
#  각 컬럼의 결측치 개수를 계산
missing_values = df.isnull().sum()
#  결측치 개수가 0보다 큰 컬럼들만 필터링하여 출력합니다.
columns_with_missing_values = missing_values[missing_values > 0]

print("## 결측값이 존재하는 컬럼 및 개수")
if columns_with_missing_values.empty:
    print("모든 컬럼의 데이터가 채워져 있습니다.")
else:
    print(columns_with_missing_values)

In [ ]:
#결측값이 존재하는 컬럼 선택 출력 확인 

df_missing = df[columns_with_missing_values.index]
print(df_missing)

In [ ]:
df[columns_with_missing_values.index]

### 중복 제거 

- unique_id 기준(0)
- Pid, title 기준(0)
- 중복되는 아이템은 없음


In [ ]:
# 중복 제거 (uniqe_id 기준으로는 중복 없음.)
df = df.drop_duplicates(subset=['uniq_id'])  
# uniq_id가 있으면 안전
df.drop_duplicates(subset=['Pid','Title'])
print(df)

In [ ]:
df.drop_duplicates(subset=['Pid','Title']) #여기도 중복은 없는 것으로 확인 

In [ ]:
df['Price'] = df['Price'].astype(str)
# price >> 문자열로 공백/ 쉼표 제고후 float
df['Price'] = df['Price'].str.replace(',', '').str.strip()
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')


### 불필요한 레코드 필터링 
- 카테고리 1, 2 기준으로 키워드 필터링 한 결과, 해당 데이터셋에서 영양제와 관련된 상품 개수는 1221->325 개로 감소함.


In [ ]:
## 화장품/ 샴푸/ 식품/ 베이비 용품 등 필터링 
non_supp_cats = ['Shampoo','Foundation','Face Wash','Utensils','Diapers']  # 예시
df = df[~df['Category 2'].isin(non_supp_cats)]
df.count

In [ ]:
# 'Category 1' 컬럼의 모든 고유값 출력
print("Category 1의 고유값:", df['Category 1'].unique())

# 'Category 2' 컬럼의 모든 고유값 출력
print("Category 2의 고유값:", df['Category 2'].unique())

# 'Category 3' 컬럼의 모든 고유값 출력
print("Category 3의 고유값:", df['Category 3'].unique())

In [ ]:
## 화이트리스트 키워드(supp_keywords)

supp_keywords = [
    # 비타민·미네랄
    'vitamin', 'multivitamin', 'multimineral', 'b1', 'b2', 'b3', 'b6',
    'b12', 'c', 'd', 'e', 'k', 'folic acid', 'niacin', 'biotin',
    'calcium', 'magnesium', 'zinc', 'iron', 'selenium', 'potassium',
    'iodine', 'trace minerals',

    # 오메가 & 필수지방산
    'omega', 'fish oil', 'krill oil', 'cod liver oil',
    'efa', 'dha', 'epa',

    # 허브·식물 추출물
    'herb', 'herbal', 'ashwagandha', 'ginseng', 'echinacea', 'turmeric',
    'curcumin', 'milk thistle', 'rhodiola', 'elderberry', 'boswellia',
    'sambucus', 'hawthorn', 'garlic', 'ginger', 'licorice', 'oregano',
    'passion flower', 'valerian', 'chamomile', 'nettle', 'schisandra',
    'astragalus',

    # 아미노산·단백질
    'amino', 'amino acid', 'l-',   # L-Arginine, L-Tyrosine 등 앞에 L-이 붙음
    'protein', 'collagen', 'peptide',

    # 프로바이오틱/소화
    'probiotic', 'prebiotic', 'lactobacillus', 'bifidus',
    'digestive enzymes', 'enzyme',

    # 항산화·기타 보조성분
    'coq10', 'ubiquinol', 'alpha lipoic acid', 'resveratrol',
    'pycnogenol', 'glutathione', 'chlorophyll', 'spirulina',
    'chlorella', 'maca', 'bee pollen', 'royal jelly',

    # 특수 목적 포뮬러
    'immune', 'immune support', 'energy formula', 'sleep formula',
    'cognitive', 'memory', 'joint', 'bone', 'liver', 'thyroid',
    'blood support', 'heart support', 'detox', 'women\'s health',
    'men\'s health', 'prenatal', 'post-natal',
    'sports supplement', 'workout', 'weight management', 'fat burner',

    # 형태·일반명
    'supplement', 'dietary', 'nutrition', 'nutrient',
    'superfood', 'greens', 'superfood blend'
]


In [ ]:
import re

def is_supplement_row(row):
    cats = " ".join([
        str(row.get('Category 2', '')).lower(),
        str(row.get('Category 3', '')).lower()
    ])
    return any(re.search(rf"\b{k}\b", cats) for k in supp_keywords)

df_supp = df[df.apply(is_supplement_row, axis=1)].copy()

print(f"필터 전 {len(df)} → 필터 후 {len(df_supp)}")

### 제대로 필터링 되었는지 확인 작업
- 멀티비타민과, 비타민 구분하여 놓았는지 

In [ ]:
import re

# 1️⃣ 화이트리스트 기반 영양제 필터
def is_supplement_row(row):
    cats = " ".join([
        str(row.get('Category 2', '')).lower(),
        str(row.get('Category 3', '')).lower()
    ])
    return any(re.search(rf"\b{k}\b", cats) for k in supp_keywords)

df_supp = df[df.apply(is_supplement_row, axis=1)].copy()

# 2️⃣ 블랙리스트 제거 (현재는 K-Beauty 하나지만 확장 가능)
black_keywords = ['k-beauty']
def not_blacklisted(row):
    cats = " ".join([
        str(row.get('Category 1', '')).lower(),
        str(row.get('Category 2', '')).lower(),
        str(row.get('Category 3', '')).lower()
    ])
    return not any(bk in cats for bk in black_keywords)

df_supp = df_supp[df_supp.apply(not_blacklisted, axis=1)].copy()

print(f"최종 필터 후 행 수 : {len(df_supp)}")

In [ ]:
# 3️⃣ 무작위 100건 추출 (중복 없이)
sample_100 = df_supp.sample(n=100, random_state=42)  # random_state는 재현성

# 4️⃣ 검토에 유용한 컬럼만 보기
cols_to_check = ['Title', 'Category 1', 'Category 2', 'Category 3', 'Description']
print(sample_100[cols_to_check].to_string(index=False))



### “파싱(parsing)”: 문자열(예: Supplement Facts 텍스트) 안에서 우리가 원하는 정보(성분명, 용량, 단위 등)를 규칙적으로 뽑아내는 작업

- 1.	parse_supplement_facts()
→ 텍스트를 줄 단위로 읽고 정규식으로 [성분, 수치, 단위] 추출.
- 2.	parse_and_flag()
→ 전체 DataFrame에 적용, parsed_ingredients와 parse_error 컬럼 추가.
- 3.	수동 검토
→ parse_error=True인 레코드만 CSV로 내보내어 직접 확인·수정.
--- 
- df_supp_checked

- parsed_ingredients: 파싱 성공 시 [{'name':…, 'amount':…, 'unit':…}, …] 리스트

- parse_error: True(실패) / False(성공)

- supplement_parse_errors.csv

사람이 직접 살펴보고 정규식 보완이나 데이터 수동 입력이 필요한 상품 목록.


In [ ]:
keyword_pattern = r'(?i)' + '|'.join([re.escape(k) for k in supp_keywords])

In [ ]:
def parse_supplement_facts(text):
    """
    Supplement Facts 문자열에서
    [성분명, 수치, 단위] 추출
    """
    results = []
    if not isinstance(text, str) or not text.strip():
        return results  # 빈 값이면 바로 실패
    
    # 예) "Vitamin C 500 mg", "Magnesium (as oxide) 250 mg"
    pattern = r'([A-Za-z0-9\-\(\) /]+?)\s+([\d.,]+)\s*(mg|mcg|µg|g|iu|IU)'
    
    for line in text.splitlines():
        m = re.search(pattern, line)
        if m:
            name = m.group(1).strip()
            amount = float(m.group(2).replace(',', ''))
            unit = m.group(3).lower()
            results.append({
                'name': name,
                'amount': amount,
                'unit': unit,
                'raw_line': line.strip()
            })
    return results

In [ ]:
def parse_and_flag_supp(df_supp):
    parsed_results = []
    parse_error_flags = []

    for text in df_supp['Supplement Facts']:
        parsed = parse_supplement_facts(text)
        parsed_results.append(parsed)
        # 파싱 결과가 없으면 True → 실패
        parse_error_flags.append(len(parsed) == 0)

    df_supp_checked = df_supp.copy()
    df_supp_checked['parsed_ingredients'] = parsed_results
    df_supp_checked['parse_error'] = parse_error_flags
    return df_supp_checked

# ✅ 실행
df_supp_checked = parse_and_flag_supp(df_supp)


In [ ]:
len(df_supp_checked)

### 파싱 실패 레코드 처리 
- HTML 태그 제거

- Proprietary / Matrix / Blend 감지

- 다중 %DV 항목 탐지

- Serving Size, Amount Per Serving 기반 구조성 여부 판단

결과: parsed, 또는 실패한 경우 실패 사유 코드 리턴 -->
1.	다양한 줄바꿈 (\n, \r\n) 혹은 공백을 제거하여 일관성 있게 처리
2.	Serving Size, Amount Per Serving, % Daily Value 등 핵심 키워드를 기준으로 텍스트를 구조화
3.	영양소 정보 블록을 정확히 추출
4.	Markdown 또는 HTML 태그, 기호 (†, ‡) 제거
5.	공란 또는 비정상 케이스에 대해 안전하게 예외 처리
--

- 텍스 자체가 난해하거나, 보충제가 아닌 상품이 섞여있어 위의 많은 시도에서 실패한 것임

-복잡한 블랜드나 표기 단위생략,불규칙한 경우는 여전히 파싱이 안된다.

# 전체 통합 정리 코드 

In [ ]:
# 1️⃣ 전체 데이터프레임: df_supp_checked 는 초기 전체 325개 데이터라고 가정
df_failed = df_supp_checked[df_supp_checked['parse_error']]  # 1차 파싱 실패

In [ ]:
# 예시: 7차 파싱 함수 (최신 버전으로 대체)
def parse_supplement_facts_v7(text):
    import re
    if not text or not isinstance(text, str) or text.strip() == "":
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}
    
    block = re.sub(r'[\n\r\t]', ' ', text.strip())
    block = re.sub(r'[†‡*%]+', '', block)
    block = re.sub(r'\s{2,}', ' ', block)

    pattern = r'([A-Za-z0-9 \-–®™\(\)\[\],\'+°]+?)\s+([\d\.,]+)\s*(mcg|mg|g|iu|IU|%)?'
    matches = re.findall(pattern, block)

    nutrients = []
    for name, amount, unit in matches:
        try:
            nutrients.append({
                "name": name.strip(),
                "amount": float(amount.replace(",", "")),
                "unit": unit or ""
            })
        except:
            continue

    return {
        "status": "success" if nutrients else "fail",
        "reason": None if nutrients else "성분 추출 실패",
        "data": nutrients if nutrients else None
    }

# ✅ 통합 적용 함수
def apply_final_parsing_v7(df):
    parsed_results = []
    parse_status = []
    fail_reasons = []

    for text in df['Supplement Facts']:
        result = parse_supplement_facts_v7(text)
        parsed_results.append(result['data'])
        parse_status.append(result['status'] == 'fail')
        fail_reasons.append(result['reason'] if result['status'] == 'fail' else None)

    df_result = df.copy()
    df_result['final7_parsed'] = parsed_results
    df_result['final7_parse_error'] = parse_status
    df_result['fail_reason_final7'] = fail_reasons

    return df_result

# ✅ 7차 파싱 실행
df_final7 = apply_final_parsing_v7(df_failed)

In [ ]:
# 전체 DataFrame 복사
df_total = df_supp_checked.copy()

# 컬럼 초기화 (초기값은 기존 결과)
df_total['final_parsed']      = df_total.get('parsed_ingredients', None)
df_total['final_parse_error'] = df_total.get('parse_error', None)
df_total['final_fail_reason'] = df_total.get('fail_reason', None)

# 7차 파싱된 인덱스를 기준으로 덮어쓰기
idx = df_final7.index
df_total.loc[idx, 'final_parsed']      = df_final7['final7_parsed']
df_total.loc[idx, 'final_parse_error'] = df_final7['final7_parse_error']
df_total.loc[idx, 'final_fail_reason'] = df_final7['fail_reason_final7']

# 성공한 경우 실패 사유 제거
df_total.loc[df_total['final_parse_error'] == False, 'final_fail_reason'] = None

In [ ]:
# 파싱 성공 / 실패 / 누락 개수 확인
total = len(df_total)
num_success = (df_total['final_parse_error'] == False).sum()
num_fail = (df_total['final_parse_error'] == True).sum()
num_na = df_total['final_parse_error'].isna().sum()

print(f"📊 전체 레코드 수: {total}")
print(f"✅ 파싱 성공: {num_success}개")
print(f"❌ 파싱 실패: {num_fail}개")
print(f"❓ 상태 미확인 (NaN): {num_na}개")


# 성분 추출 실패 파일 만들기 위해 추가한 코드 

In [ ]:
df_total.columns
#df_total['fail_reason']

In [ ]:
print("전체 개수:", len(df_total['final_fail_reason']))
print("실패 사유가 있는 행 수:", df_total['final_fail_reason'].notna().sum())
print("실패 사유가 없는 행 수 (NaN):", df_total['final_fail_reason'].isna().sum())
print(df_total['final_fail_reason'].value_counts())

In [61]:
import pandas as pd

# 1️⃣ 성분 추출 실패한 데이터만 필터링
df_failed_extracted = df_total[df_total['final_fail_reason'] == '성분 추출 실패'].copy()

# 2️⃣ 필요한 컬럼만 선택해서 확인
columns_to_save = [
    "Title",
    "Category 1", "Category 2", "Category 3",
    "Brand",
    "Supplement Facts",
    "Description",
    "final_fail_reason"
]

df_failed_export = df_failed_extracted[columns_to_save]

# 3️⃣ CSV로 저장
df_failed_export.to_csv("supplement_failed_성분추출실패.csv", index=False, encoding='utf-8-sig')

print("📁 supplement_failed_성분추출실패.csv 저장 완료!")

📁 supplement_failed_성분추출실패.csv 저장 완료!


In [ ]:
# 인덱스 정렬 후 병합 (권장)
df_total['fail_reason'] = df_final7['fail_reason_final7'].reindex(df_total.index)

In [ ]:
if 'fail_reason' not in df_total.columns:
    print("⚠️ 'fail_reason' 컬럼이 없습니다. 먼저 추가해 주세요.")
else:
    print(df_total['fail_reason'].value_counts())

In [ ]:
# 성분 추출 실패한 92개 개선 
def parse_supplement_facts_v8(text: str) -> dict:
    import re
    if not text or not isinstance(text, str) or text.strip() == "":
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    block = re.sub(r'[\n\r\t]', ' ', text.strip())
    block = re.sub(r'[†‡*%]+', '', block)
    block = re.sub(r'\s{2,}', ' ', block)

    # 좀 더 자유롭게 괄호와 단위 허용
    pattern = r'([A-Za-z0-9 \-–®™\(\)\[\],\'°©®]+?)\s*[:\-]?\s*([\d\.,]+)\s*(mcg|mg|g|iu|IU|ml|%)?'

    matches = re.findall(pattern, block)

    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    nutrients = []
    for name, amount, unit in matches:
        try:
            nutrients.append({
                "name": name.strip(),
                "amount": float(amount.replace(',', '')),
                "unit": unit or ''
            })
        except:
            continue

    return {
        "status": "success" if nutrients else "fail",
        "reason": None if nutrients else "성분 추출 실패",
        "data": nutrients,
        "count": len(nutrients)
    }

In [ ]:
df_fail = df_total[df_total['final_parse_error']]
df_retry8 = df_fail.copy()

# 새로 파싱 시도
results = df_retry8['Supplement Facts'].apply(parse_supplement_facts_v8)

df_retry8['final8_parsed'] = results.map(lambda x: x['data'])
df_retry8['final8_parse_error'] = results.map(lambda x: x['status'] == 'fail')
df_retry8['fail_reason_final8'] = results.map(lambda x: x['reason'])

In [ ]:
# 먼저 index가 일치하는지 확인 (안하면 오류납니다!)
df_retry8 = df_retry8.copy()
df_retry8 = df_retry8[['final8_parsed', 'final8_parse_error', 'fail_reason_final8']]

# 원본 df_total에 있는 실패 데이터의 인덱스 기준으로 업데이트
df_total.loc[df_retry8.index, 'parsed_data'] = df_retry8['final8_parsed']
df_total.loc[df_retry8.index, 'parse_error'] = df_retry8['final8_parse_error']
df_total.loc[df_retry8.index, 'fail_reason'] = df_retry8['fail_reason_final8']

In [ ]:
# 파싱 성공/실패/NaN 통계 요약
total = len(df_total)
num_success = (df_total['parse_error'] == False).sum()
num_fail = (df_total['parse_error'] == True).sum()
num_na = df_total['parse_error'].isna().sum()

print(f"📊 전체 레코드 수: {total}")
print(f"✅ 파싱 성공: {num_success}개")
print(f"❌ 파싱 실패: {num_fail}개")
print(f"❓ 상태 미확인 (NaN): {num_na}개") 

In [ ]:
fail_summary = df_total[df_total['parse_error'] == True]['fail_reason'].value_counts()
print("\n📉 파싱 실패 사유 분포:")
print(fail_summary)

In [ ]:
# 실패한 레코드 중에서 실패 사유가 비어 있는 (NaN) 데이터
df_fail = df_total[df_total['parse_error'] == True]
df_fail_nan_reason = df_fail[df_fail['fail_reason'].isna()]

print(f"❓ 실패했지만 실패 사유가 없는 레코드 수: {len(df_fail_nan_reason)}개")

In [ ]:
# 실패 사유가 없는 데이터만 저장
df_fail_nan_reason.to_csv('supplement_parsing_failed_reason_missing.csv', index=False)
print("📁 supplement_parsing_failed_reason_missing.csv 파일로 저장 완료")

In [ ]:
# 파싱 실패한 106개 레코드만 필터링
df_fail = df_total[df_total['parse_error'] == True]

# 컬럼 목록 출력
print("📋 파싱 실패한 데이터프레임의 컬럼 목록:")
print(df_fail.columns.tolist())

In [ ]:
print(df_total.columns)

In [ ]:
df_parsed = df_total[df_total['final_parse_error'] == False]

In [ ]:
# 즉, **parse_error 컬럼을 기준으로 했기 때문에 219개**로 나왔던 것이고,
# final_parse_error 기준은 214개입니다

len(df_parsed)

In [ ]:
# ✅ 최종 파싱 성공한 데이터만 필터링
df_parsed = df_total[df_total['final_parse_error'] == False]

# ✅ CSV 파일로 저장
df_parsed.to_csv("parsed_supplements_final.csv", index=False)
print("✅ 파싱된 데이터가 'parsed_supplements_final.csv'로 저장되었습니다.")

In [ ]:
df_parsed

In [ ]:
# 예: 인덱스 6번의 전체 데이터 보기
import pprint
pprint.pprint(df_parsed['final_parsed'].iloc[0])

In [ ]:
# ✅ parsed_data 컬럼 제거
df_parsed.drop(columns=['parsed_data'], inplace=True)

In [ ]:
df_parsed.to_csv("parsed_supplements_final_cleaned.csv", index=False)
print("✅ cleaned 파일 저장 완료")

In [ ]:
len(df_parsed)

## 컬럼명 정리
final_parsed
✅ 최종 파싱 결과 리스트 ([{name, amount, unit, ...}, ...] 형태). 실제로 Supplement Facts를 정규표현식으로 파싱한 결과입니다.
final_parse_error
✅ 파싱 성공 여부를 담은 True/False 값. True면 파싱 실패, False면 성공입니다.
final_fail_reason
✅ 파싱 실패 이유를 설명하는 텍스트. 예: "성분 추출 실패", "공란 또는 타입 오류" 등
fail_reason
(중간 버전에서 사용하던) 파싱 실패 사유. 지금은 final_fail_reason이 최신이므로 이건 거의 NaN 상태일 수 있어요.
parsed_data
❌ 초기 파싱 때 쓰던 결과 컬럼인데, 지금은 쓰지 않아요. 최종 결과는 final_parsed로 대체되었습니다.


In [ ]:
df_parsed.columns

In [ ]:
print(df_parsed['fail_reason'].value_counts(dropna=False))

In [ ]:
df_fail

# 8차 파싱 함수


In [62]:
import re

def parse_supplement_facts_v8(text: str) -> dict:
    if not text or not isinstance(text, str) or text.strip() == "":
        return {"status": "fail", "reason": "공란 또는 타입 오류", "data": None}

    # 1️⃣ 특수문자 및 공백 정리
    block = re.sub(r'[\n\r\t]', ' ', text.strip())  # 줄바꿈 제거
    block = re.sub(r'[†‡*%•●◆…–—✔️→→▶→➡️★]', '', block)  # 특수문자 제거
    block = re.sub(r'\s{2,}', ' ', block)  # 다중 공백 정리
    block = re.sub(r'(\d)([A-Za-z])', r'\1 \2', block)  # 숫자와 문자가 붙어있는 경우 분리
    block = re.sub(r'([A-Za-z])(\d)', r'\1 \2', block)  # 문자와 숫자가 붙어있는 경우 분리

    # 2️⃣ 정규식 개선: 성분명에는 괄호, 대괄호, 상표까지 허용
    # 예: "Zinc (as Zinc Gluconate)" 또는 "Selenium [from Selenium Yeast]"
    pattern = r'([A-Za-z0-9 \-–®™\(\)\[\],\'°©®\/&]+?)\s*[:\-]?\s*([\d\.,]+)\s*(mg|mcg|g|IU|iu|ml|billion CFU|CFU|ALU|HUT|XU|DP|SU|CU|FIP|DPPU|%)?'

    matches = re.findall(pattern, block)

    if not matches:
        return {"status": "fail", "reason": "성분 추출 실패", "data": None}

    nutrients = []
    for name, amount, unit in matches:
        try:
            nutrients.append({
                "name": name.strip(),
                "amount": float(amount.replace(',', '')),
                "unit": unit or '',
                "raw_line": block  # 원본 라인 전체 저장
            })
        except:
            continue

    return {
        "status": "success" if nutrients else "fail",
        "reason": None if nutrients else "성분 추출 실패",
        "data": nutrients if nutrients else None,
        "count": len(nutrients)
    }

In [63]:
# 실패한 데이터만 추출
df_failed = df_total[df_total['final_parse_error']]

# 새 파싱 적용
results = df_failed['Supplement Facts'].apply(parse_supplement_facts_v8)

# 결과 반영
df_failed['final8_parsed'] = results.map(lambda x: x['data'])
df_failed['final8_parse_error'] = results.map(lambda x: x['status'] == 'fail')
df_failed['fail_reason_final8'] = results.map(lambda x: x['reason'])

# 병합
df_total = df_total.copy()
df_total.loc[df_failed.index, 'final8_parsed'] = df_failed['final8_parsed']
df_total.loc[df_failed.index, 'final8_parse_error'] = df_failed['final8_parse_error']
df_total.loc[df_failed.index, 'fail_reason_final8'] = df_failed['fail_reason_final8']

/var/folders/98/vj3q9b254k79y681ng12dddh0000gn/T/ipykernel_96580/665848918.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_failed['final8_parsed'] = results.map(lambda x: x['data'])
/var/folders/98/vj3q9b254k79y681ng12dddh0000gn/T/ipykernel_96580/665848918.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_failed['final8_parse_error'] = results.map(lambda x: x['status'] == 'fail')
/var/folders/98/vj3q9b254k79y681ng12dddh0000gn/T/ipykernel_96580/665848918.py:10: SettingWithCopyWarning: 
A value is

In [64]:
# 성공: 파싱 에러가 False
num_success = (df_total['final8_parse_error'] == False).sum()

# 실패: 파싱 에러가 True
num_fail = (df_total['final8_parse_error'] == True).sum()

# 누락: 값 자체가 NaN (파싱 시도되지 않은 경우 포함 가능)
num_nan = df_total['final8_parse_error'].isna().sum()

print(f"📦 전체 레코드 수: {len(df_total)}")
print(f"✅ 파싱 성공: {num_success}개")
print(f"❌ 파싱 실패: {num_fail}개")
print(f"❓ 상태 미확인 (NaN): {num_nan}개")

📦 전체 레코드 수: 325
✅ 파싱 성공: 92개
❌ 파싱 실패: 19개
❓ 상태 미확인 (NaN): 214개


In [65]:
# 파싱 성공 데이터 저장
df_success = df_total[df_total['final8_parse_error'] == False]
df_success.to_csv("supplements_parsed_success_final8.csv", index=False)

# # 파싱 실패 데이터 저장
# df_fail = df_total[df_total['final8_parse_error'] == True]
# df_fail.to_csv("supplements_parsed_failed_final8.csv", index=False)

# # 누락 (NaN) 데이터 저장
# df_na = df_total[df_total['final8_parse_error'].isna()]
# df_na.to_csv("supplements_parsed_unknown_final8.csv", index=False)

In [67]:
import json

def df_to_rag_json(df, parsed_col='final_parsed'):
    rag_data = []

    for _, row in df.iterrows():
        entry = {
            "title": row["Title"],
            "category": [row.get("Category 1", ""), row.get("Category 2", ""), row.get("Category 3", "")],
            "brand": row.get("Brand", ""),
            "supplement_facts": row.get(parsed_col, []),
            "description": row.get("Description", ""),
            "tags": list({
                *(str(row.get("Category 1", "")).split()),
                *(str(row.get("Category 2", "")).split()),
                *(str(row.get("Category 3", "")).split()),
                *(i["name"] for i in row.get(parsed_col, []) if isinstance(i, dict))
            })
        }
        rag_data.append(entry)

    return rag_data

# 변환 및 저장
rag_json = df_to_rag_json(df_success)

with open("supplements_rag_data_final8.json", "w", encoding="utf-8") as f:
    json.dump(rag_json, f, ensure_ascii=False, indent=2)

print("📦 'supplements_rag_data_final8.json' 저장 완료")

TypeError: 'NoneType' object is not iterable

In [68]:
import json

def df_to_rag_json(df, parsed_col='final_parsed'):
    rag_data = []

    for _, row in df.iterrows():
        parsed_data = row.get(parsed_col)
        if not isinstance(parsed_data, list):
            parsed_data = []  # None 또는 문자열이면 빈 리스트로

        entry = {
            "title": row["Title"],
            "category": [row.get("Category 1", ""), row.get("Category 2", ""), row.get("Category 3", "")],
            "brand": row.get("Brand", ""),
            "supplement_facts": parsed_data,
            "description": row.get("Description", ""),
            "tags": list({
                *(str(row.get("Category 1", "")).split()),
                *(str(row.get("Category 2", "")).split()),
                *(str(row.get("Category 3", "")).split()),
                *(i["name"] for i in parsed_data if isinstance(i, dict) and "name" in i)
            })
        }
        rag_data.append(entry)

    return rag_data

# ✅ JSON 생성 및 저장
rag_json = df_to_rag_json(df_success)

with open("supplements_rag_data_final8.json", "w", encoding="utf-8") as f:
    json.dump(rag_json, f, ensure_ascii=False, indent=2)

print("✅ JSON 파일 저장 완료: supplements_rag_data_final8.json")

✅ JSON 파일 저장 완료: supplements_rag_data_final8.json
